In [1]:
import sys
sys.path
sys.path.append(r"C:\Users\yanpe\.env\Lib\site-packages")
print(sys.path)

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_

In [2]:
import os
import open3d as o3d
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.typing import WITH_TORCH_CLUSTER
from pyg_pointnet2 import PyGPointNet2, PyGPointNet2NoColor

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch_geometric\typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch_geometric\typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
# Empty the CUDA cache
torch.cuda.empty_cache()

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = PyGPointNet2(num_classes=13).to(device)
model = PyGPointNet2NoColor(num_classes=13).to(device) # colorless model


In [7]:
# File path to the pre-trained model
script_dir = os.path.dirname(os.getcwd())
#model_file_path= os.path.join(script_dir, '.', 'docs', 'pointnet2_s3dis_transform_seg_x6_45_checkpoint.pth')
model_file_path= os.path.join(script_dir, '.', 'docs', 'pointnet2_s3dis_colorless_seg_x3_45_checkpoint.pth') # colorless model
# pointnet2_s3dis_transform_seg_x6_27_checkpoint
# pointnet2_smartlab_sim_bw_seg_x6_0413_checkpoint
# pointnet2_s3dis_colorless_seg_x3_45_checkpoint.pth


In [4]:
model_file_path = "checkpoints/pointnet2_s3dis_colorless_seg_x3_45_checkpoint.pth"

In [7]:
# Load the checkpoint dictionary
checkpoint = torch.load(model_file_path, map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']

# print(model_state_dict)

In [8]:
model.load_state_dict(model_state_dict)
model.eval()

PyGPointNet2NoColor(
  (sa1_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(6, 64, 64, 128), global_nn=None)
  )
  (sa2_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(131, 128, 128, 256), global_nn=None)
  )
  (sa3_module): GlobalSAModule(
    (nn): MLP(259, 256, 512, 1024)
  )
  (fp3_module): FPModule(
    (nn): MLP(1280, 256, 256)
  )
  (fp2_module): FPModule(
    (nn): MLP(384, 256, 128)
  )
  (fp1_module): FPModule(
    (nn): MLP(131, 128, 128, 128)
  )
  (mlp): MLP(128, 128, 128, 13)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=13, bias=True)
)

In [9]:
# Pcd files
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/Smartlab-2024-04-05_10-58-26_colour_cleaned.pcd"
pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.pcd"
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.ply"
pcd = o3d.io.read_point_cloud(pcd_path)

In [8]:
print(len(pcd.points))
print(len(pcd.colors))
print(len(pcd.normals))

35966772
35966772
0


In [9]:
def array_range(array):
    # Get min and max for each column
    min_values = np.min(array, axis=0)
    max_values = np.max(array, axis=0)

    # Display ranges for each column
    for i, (min_val, max_val) in enumerate(zip(min_values, max_values), start=1):
        print(f"Column {i}: min = {min_val}, max = {max_val}")

In [54]:
print(array_range(np.array(pcd.points)))

Column 1: min = -1.037500023841858, max = 9.753499984741211
Column 2: min = -4.067580223083496, max = 7.109419822692871
Column 3: min = -1.562000036239624, max = 1.8949999809265137
None


In [49]:
#if not pcd.has_normals():
#   pcd.estimate_normals()

In [10]:
# Downsample the point cloud with a voxel of 0.02
downpcd = pcd.voxel_down_sample(voxel_size=0.03)

print(len(downpcd.points))

866885


In [26]:
o3d.visualization.draw_geometries([downpcd], point_show_normal=False)

In [11]:
def normalize_points(points):
    # Step 1: Center the points around the origin
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid

    # Step 2: Scale to fit within the [0, 1] interval
    min_vals = np.min(centered_points, axis=0)
    max_vals = np.max(centered_points, axis=0)
    scale = max_vals - min_vals

    # Avoid division by zero in case of flat dimension
    scale[scale == 0] = 1  # Set zero scales to 1 to keep that dimension as 0.5 after normalization

    normalized_points = (centered_points - min_vals) / scale

    return normalized_points

normalized = normalize_points(np.array(downpcd.points))

In [11]:
def normalize_points_corner(points):
    # Step 1: Shift points so that the minimum x, y, z becomes the origin.
    min_vals = np.min(points, axis=0)
    shifted_points = points - min_vals  # Now the lower bound is (0,0,0)
    
    # Step 2: Compute the scaling factors from the shifted points.
    # The maximum after shifting represents the range in each dimension.
    max_vals = np.max(shifted_points, axis=0)
    scale = max_vals.copy()
    
    # Avoid division by zero if any dimension is flat.
    scale[scale == 0] = 1
    
    # Normalize the shifted points to the [0, 1] interval.
    normalized_points = shifted_points / scale

    return normalized_points

normalized = normalize_points_corner(np.array(downpcd.points))

In [12]:
# Extract coordinates and colors from the point cloud
down_points = torch.tensor(np.array(downpcd.points), dtype=torch.float32)  
down_colors = torch.tensor(np.array(downpcd.colors), dtype=torch.float32)
down_normalized = torch.tensor(normalized, dtype=torch.float32)

In [13]:
# Concatenate coordinates and colors to form the input features
x = torch.cat([down_colors, down_normalized], dim=1) # 6 (x)


In [17]:
# Create a Data object with x (6 features) and pos (coordinates)
#data = Data(x=x, pos=down_points)

# Create a Data object with x (3 features) and pos (coordinates)
data = Data(x=down_normalized, pos=down_points)

#data = data.to(device)

#pre_transform = T.NormalizeScale()
#data = pre_transform(data)

In [88]:
import os
os.cpu_count()

22

In [18]:
# If you have only one point cloud
dataset = [data]  # List of Data objects

num_workers = 20
batch_size = 32

# Create a DataLoader (batch_size can be adjusted as needed)
custom_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [ ]:
import time

model.eval()

all_labels = []
start_time = time.time()

# Profiler removed from this block
with torch.no_grad(), torch.amp.autocast("cuda"):
    for data in custom_loader:
        data = data.to(device, non_blocking=True)
        predictions = model(data)
        labels = predictions.argmax(dim=-1)
        #all_labels.append(labels)
    end_time = time.time()
    print(f"Total inference time: {end_time - start_time:.4f} seconds")
        
# Concatenate all labels into one tensor
#combined_labels = torch.cat(all_labels, dim=0)

# Compute unique labels and their counts once over the whole tensor
#unique_labels, label_counts = torch.unique(combined_labels, return_counts=True)
#result_labels = torch.stack((unique_labels, label_counts), dim=1).cpu()

#print("Label counts:")
#print(result_labels)

In [19]:
import time

model.eval()

import torch.profiler

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    record_shapes=True,
) as prof:

    with torch.no_grad():
        start_time = time.time()
        for data in custom_loader:
            data = data.to(device)
            with torch.amp.autocast("cuda"):
                predictions = model(data)
            labels = predictions.argmax(dim=-1)
            # Process the labels as needed
            #labels_arr = labels.cpu().numpy()
            # Count occurrences of labels
            unique_labels, label_counts = torch.unique(labels, return_counts=True)
            # Combine and print
            #result_labels = np.array(list(zip(unique_labels, label_counts)))
            #print("Label counts:")
            #for label, count in zip(unique_labels, label_counts):
            #    print(f"Label {label.item()}: {count.item()}")
            result_labels = torch.stack((unique_labels, label_counts), dim=1).cpu()
            print("Label counts:")
            print(result_labels)
        end_time = time.time()
        print(f"Total inference time: {end_time - start_time:.4f} seconds")  
    
print(prof.key_averages().table(sort_by="cuda_time_total"))



Label counts:
tensor([[     0,  94891],
        [     1, 127005],
        [     2, 159255],
        [     3, 114407],
        [     4,  25909],
        [     6, 117425],
        [     7,   3835],
        [     8,   7415],
        [    10,     34],
        [    12, 216709]])
Total inference time: 257.6336 seconds
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                     torch_cluster::fps         0.00%       2.464ms        

batch_size 32, num_workers 16: prediction 7min

batch_size = 32, num_workers 20: prediction 5min

batch_size = 32, num_workers 20, pin_memory=True: prediction 4.5min



In [20]:

# Define the number of classes in your model's predictions
num_classes = 13  # Adjust based on your number of classes

# class_names = ('ceiling', 'floor', 'wall', 'beam', 'column', 'window', 'door',
#               'table', 'chair', 'sofa', 'bookcase', 'board', 'clutter')

# Define a fixed color map for 13 labels
color_map = np.array([
    [1.0, 0.0, 0.0],  # Label 0: Red,  'ceiling'
    [0.0, 1.0, 0.0],  # Label 1: Green, 'floor'
    [0.0, 0.0, 1.0],  # Label 2: Blue,  'wall'
    [1.0, 1.0, 0.0],  # Label 3: Yellow, 'beam'
    [1.0, 0.0, 1.0],  # Label 4: Magenta, 'column'
    [0.0, 1.0, 1.0],  # Label 5: Cyan, 'window'
    [0.5, 0.5, 0.5],  # Label 6: Gray, 'door'
    [1.0, 0.5, 0.0],  # Label 7: Orange, 'table'
    [0.5, 0.0, 1.0],  # Label 8: Purple, 'chair'
    [0.5, 1.0, 0.5],  # Label 9: Light Green, 'sofa'
    [0.5, 0.5, 1.0],  # Label 10: Light Blue, 'bookcase'
    [1.0, 0.5, 0.5],  # Label 11: Pink, 'board'
    [0.0, 0.0, 0.0]   # Label 12: Black, 'clutter'
])

In [21]:
# Assuming `labels` is the tensor containing predicted labels for each point
predicted_colors = color_map[labels.cpu().numpy()]  # Shape: [num_points, 3]


In [18]:
print(len(predicted_colors))

1950239


In [22]:
# Assuming `pcd` is your Open3D point cloud object
downpcd.colors = o3d.utility.Vector3dVector(predicted_colors)

In [23]:
# Visualize the point cloud with colored labels
o3d.visualization.draw_geometries([downpcd])

In [24]:
# Save the point cloud to a file
save_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/labelled/Smartlab_aalto_pcd_center_norm_label_pointnet2_x3_0.03_20250415.ply"
o3d.io.write_point_cloud(save_path, downpcd)


True

In [50]:
del down_points
del down_colors
torch.cuda.empty_cache()